In [1]:
"""
Healthcare Data Analysis Project
Analyzing patient treatment outcomes, costs, and resource utilization
"""

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import json

# Set style for better-looking plots
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

In [2]:
# ==================== STEP 1: LOAD & EXPLORE DATA ====================
print("=" * 60)
print("HEALTHCARE DATA ANALYSIS PROJECT")
print("=" * 60)

# Load the dataset
df = pd.read_csv('healthcare_data.csv')

print("\n📊 Dataset Overview:")
print(f"Total Patients: {len(df)}")
print(f"Date Range: {df['admission_date'].min()} to {df['discharge_date'].max()}")
print(f"\nColumns: {', '.join(df.columns)}")

# Basic statistics
print("\n" + "=" * 60)
print("BASIC STATISTICS")
print("=" * 60)
print(df.describe())

# Check for missing values
print("\n🔍 Missing Values:")
print(df.isnull().sum())

HEALTHCARE DATA ANALYSIS PROJECT

📊 Dataset Overview:
Total Patients: 70
Date Range: 2024-01-05 to 2024-07-05

Columns: patient_id, age, gender, condition, treatment_type, admission_date, discharge_date, length_of_stay, total_cost, treatment_success, readmitted, insurance_type, doctor_visits

BASIC STATISTICS
             age  length_of_stay    total_cost  doctor_visits
count  70.000000       70.000000     70.000000      70.000000
mean   52.857143        5.128571  12957.142857       3.914286
std    14.898970        1.817160   4767.053945       1.576483
min    28.000000        3.000000   7200.000000       2.000000
25%    39.250000        4.000000   8950.000000       3.000000
50%    55.500000        4.000000  11450.000000       3.000000
75%    66.000000        7.000000  17025.000000       5.000000
max    76.000000        8.000000  23100.000000       7.000000

🔍 Missing Values:
patient_id           0
age                  0
gender               0
condition            0
treatment_type      

In [3]:
# ==================== STEP 2: DATA CLEANING ====================
print("\n" + "=" * 60)
print("DATA CLEANING")
print("=" * 60)

# Convert dates to datetime
df['admission_date'] = pd.to_datetime(df['admission_date'])
df['discharge_date'] = pd.to_datetime(df['discharge_date'])

# Extract month for trend analysis
df['admission_month'] = df['admission_date'].dt.to_period('M')

# Binary encoding for success
df['success_binary'] = df['treatment_success'].map({'Yes': 1, 'No': 0})
df['readmitted_binary'] = df['readmitted'].map({'Yes': 1, 'No': 0})

print("✅ Data cleaned and prepared!")
print(f"Date columns converted to datetime format")
print(f"Added binary columns for statistical analysis")


DATA CLEANING
✅ Data cleaned and prepared!
Date columns converted to datetime format
Added binary columns for statistical analysis


In [4]:
# ==================== STEP 3: KEY METRICS ANALYSIS ====================
print("\n" + "=" * 60)
print("KEY PERFORMANCE INDICATORS")
print("=" * 60)

# Overall metrics
total_patients = len(df)
avg_length_stay = df['length_of_stay'].mean()
avg_cost = df['total_cost'].mean()
success_rate = (df['success_binary'].sum() / total_patients) * 100
readmission_rate = (df['readmitted_binary'].sum() / total_patients) * 100

print(f"\n📈 Hospital Performance:")
print(f"   • Total Patients Treated: {total_patients}")
print(f"   • Average Length of Stay: {avg_length_stay:.1f} days")
print(f"   • Average Treatment Cost: ${avg_cost:,.2f}")
print(f"   • Treatment Success Rate: {success_rate:.1f}%")
print(f"   • Readmission Rate: {readmission_rate:.1f}%")



KEY PERFORMANCE INDICATORS

📈 Hospital Performance:
   • Total Patients Treated: 70
   • Average Length of Stay: 5.1 days
   • Average Treatment Cost: $12,957.14
   • Treatment Success Rate: 92.9%
   • Readmission Rate: 4.3%


In [5]:

# ==================== STEP 4: CONDITION ANALYSIS ====================
print("\n" + "=" * 60)
print("ANALYSIS BY MEDICAL CONDITION")
print("=" * 60)

condition_stats = df.groupby('condition').agg({
    'patient_id': 'count',
    'length_of_stay': 'mean',
    'total_cost': 'mean',
    'success_binary': 'mean',
    'readmitted_binary': 'mean'
}).round(2)

condition_stats.columns = ['Patient_Count', 'Avg_Stay_Days', 'Avg_Cost', 'Success_Rate', 'Readmission_Rate']
condition_stats['Success_Rate'] = (condition_stats['Success_Rate'] * 100).round(1)
condition_stats['Readmission_Rate'] = (condition_stats['Readmission_Rate'] * 100).round(1)

print("\n", condition_stats)

# Find best and worst performing conditions
best_condition = condition_stats['Success_Rate'].idxmax()
worst_condition = condition_stats['Success_Rate'].idxmin()
most_expensive = condition_stats['Avg_Cost'].idxmax()

print(f"\n🏆 Key Findings:")
print(f"   • Best Success Rate: {best_condition} ({condition_stats.loc[best_condition, 'Success_Rate']:.1f}%)")
print(f"   • Needs Improvement: {worst_condition} ({condition_stats.loc[worst_condition, 'Success_Rate']:.1f}%)")
print(f"   • Most Expensive: {most_expensive} (${condition_stats.loc[most_expensive, 'Avg_Cost']:,.2f})")



ANALYSIS BY MEDICAL CONDITION

                Patient_Count  Avg_Stay_Days  Avg_Cost  Success_Rate  \
condition                                                             
Diabetes                  14           4.07   8921.43         100.0   
Heart Disease             15           7.40  17833.33          67.0   
Neurological               9           7.56  20688.89         100.0   
Orthopedic                14           3.00  12371.43         100.0   
Respiratory               18           4.50   8622.22         100.0   

               Readmission_Rate  
condition                        
Diabetes                    0.0  
Heart Disease              20.0  
Neurological                0.0  
Orthopedic                  0.0  
Respiratory                 0.0  

🏆 Key Findings:
   • Best Success Rate: Diabetes (100.0%)
   • Needs Improvement: Heart Disease (67.0%)
   • Most Expensive: Neurological ($20,688.89)


In [6]:
# ==================== STEP 5: TREATMENT TYPE ANALYSIS ====================
print("\n" + "=" * 60)
print("TREATMENT TYPE EFFECTIVENESS")
print("=" * 60)

treatment_stats = df.groupby('treatment_type').agg({
    'patient_id': 'count',
    'total_cost': 'mean',
    'success_binary': 'mean',
    'length_of_stay': 'mean'
}).round(2)

treatment_stats.columns = ['Patient_Count', 'Avg_Cost', 'Success_Rate', 'Avg_Stay']
treatment_stats['Success_Rate'] = (treatment_stats['Success_Rate'] * 100).round(1)

print("\n", treatment_stats)



TREATMENT TYPE EFFECTIVENESS

                 Patient_Count  Avg_Cost  Success_Rate  Avg_Stay
treatment_type                                                 
Medication                 29  10979.31          93.0      5.00
Surgery                    23  17386.96          87.0      5.91
Therapy                    18  10483.33         100.0      4.33


In [7]:
print("\n" + "=" * 60)
print("AGE GROUP ANALYSIS")
print("=" * 60)

# Create age groups
df['age_group'] = pd.cut(df['age'], bins=[0, 35, 50, 65, 100], 
                         labels=['Young (18-35)', 'Middle (36-50)', 'Senior (51-65)', 'Elderly (66+)'])

age_analysis = df.groupby('age_group').agg({
    'patient_id': 'count',
    'total_cost': 'mean',
    'length_of_stay': 'mean',
    'success_binary': 'mean'
}).round(2)

age_analysis.columns = ['Patient_Count', 'Avg_Cost', 'Avg_Stay', 'Success_Rate']
age_analysis['Success_Rate'] = (age_analysis['Success_Rate'] * 100).round(1)

print("\n", age_analysis)

# Correlation between age and cost
age_cost_corr = df['age'].corr(df['total_cost'])
print(f"\n📊 Age-Cost Correlation: {age_cost_corr:.3f}")
if age_cost_corr > 0.3:
    print("   → Moderate positive correlation: Older patients tend to have higher costs")



AGE GROUP ANALYSIS

                 Patient_Count  Avg_Cost  Avg_Stay  Success_Rate
age_group                                                      
Young (18-35)              12   8716.67      3.92         100.0
Middle (36-50)             19  11373.68      4.58         100.0
Senior (51-65)             20  13225.00      5.30          90.0
Elderly (66+)              19  16936.84      6.26          84.0

📊 Age-Cost Correlation: 0.608
   → Moderate positive correlation: Older patients tend to have higher costs


In [8]:
# ==================== STEP 7: INSURANCE ANALYSIS ====================
print("\n" + "=" * 60)
print("INSURANCE TYPE ANALYSIS")
print("=" * 60)

insurance_stats = df.groupby('insurance_type').agg({
    'patient_id': 'count',
    'total_cost': 'mean',
    'length_of_stay': 'mean'
}).round(2)

insurance_stats.columns = ['Patient_Count', 'Avg_Cost', 'Avg_Stay']
print("\n", insurance_stats)



INSURANCE TYPE ANALYSIS

                 Patient_Count  Avg_Cost  Avg_Stay
insurance_type                                   
Medicaid                    8   9375.00      4.12
Medicare                   27  16359.26      6.00
Private                    35  11151.43      4.69


In [9]:
# ==================== STEP 8: MONTHLY TRENDS ====================
print("\n" + "=" * 60)
print("MONTHLY ADMISSION TRENDS")
print("=" * 60)

monthly_trends = df.groupby('admission_month').agg({
    'patient_id': 'count',
    'total_cost': 'mean'
}).round(2)

monthly_trends.columns = ['Admissions', 'Avg_Cost']
print("\n", monthly_trends)



MONTHLY ADMISSION TRENDS

                  Admissions  Avg_Cost
admission_month                      
2024-01                  10  12690.00
2024-02                  12  12750.00
2024-03                  11  13654.55
2024-04                  12  13208.33
2024-05                  12  12291.67
2024-06                  12  13616.67
2024-07                   1   7500.00


In [10]:
# ==================== STEP 9: COST OPTIMIZATION INSIGHTS ====================
print("\n" + "=" * 60)
print("COST OPTIMIZATION OPPORTUNITIES")
print("=" * 60)

# Find expensive conditions with long stays
expensive_long_stay = df[(df['total_cost'] > df['total_cost'].median()) & 
                         (df['length_of_stay'] > df['length_of_stay'].median())]

print(f"\n💰 High-Cost, Long-Stay Cases: {len(expensive_long_stay)} patients")
print(f"   Average Cost: ${expensive_long_stay['total_cost'].mean():,.2f}")
print(f"   Average Stay: {expensive_long_stay['length_of_stay'].mean():.1f} days")
print(f"   Top Conditions: {expensive_long_stay['condition'].value_counts().head(3).to_dict()}")



COST OPTIMIZATION OPPORTUNITIES

💰 High-Cost, Long-Stay Cases: 24 patients
   Average Cost: $18,904.17
   Average Stay: 7.5 days
   Top Conditions: {'Heart Disease': 15, 'Neurological': 9}


In [11]:
# ==================== STEP 10: STATISTICAL INSIGHTS ====================
print("\n" + "=" * 60)
print("STATISTICAL CORRELATIONS")
print("=" * 60)

# Correlation matrix
correlations = df[['age', 'length_of_stay', 'total_cost', 'doctor_visits', 'success_binary']].corr()
print("\n", correlations.round(3))

print("\n🔍 Key Correlations:")
print(f"   • Length of Stay ↔ Cost: {correlations.loc['length_of_stay', 'total_cost']:.3f}")
print(f"   • Age ↔ Length of Stay: {correlations.loc['age', 'length_of_stay']:.3f}")
print(f"   • Doctor Visits ↔ Cost: {correlations.loc['doctor_visits', 'total_cost']:.3f}")



STATISTICAL CORRELATIONS

                   age  length_of_stay  total_cost  doctor_visits  \
age             1.000           0.466       0.608          0.423   
length_of_stay  0.466           1.000       0.815          0.980   
total_cost      0.608           0.815       1.000          0.799   
doctor_visits   0.423           0.980       0.799          1.000   
success_binary -0.284          -0.380      -0.292         -0.299   

                success_binary  
age                     -0.284  
length_of_stay          -0.380  
total_cost              -0.292  
doctor_visits           -0.299  
success_binary           1.000  

🔍 Key Correlations:
   • Length of Stay ↔ Cost: 0.815
   • Age ↔ Length of Stay: 0.466
   • Doctor Visits ↔ Cost: 0.799


In [12]:
# ==================== STEP 11: SAVE RESULTS FOR DASHBOARD ====================
print("\n" + "=" * 60)
print("GENERATING DASHBOARD DATA")
print("=" * 60)

# Prepare data for dashboard
dashboard_data = {
    'summary': {
        'total_patients': int(total_patients),
        'avg_stay': round(float(avg_length_stay), 1),
        'avg_cost': round(float(avg_cost), 2),
        'success_rate': round(float(success_rate), 1),
        'readmission_rate': round(float(readmission_rate), 1)
    },
    'by_condition': condition_stats.reset_index().to_dict('records'),
    'by_treatment': treatment_stats.reset_index().to_dict('records'),
    'by_age_group': age_analysis.reset_index().to_dict('records'),
    'monthly_trends': [
        {'month': str(idx), 'admissions': int(row['Admissions']), 'avg_cost': round(float(row['Avg_Cost']), 2)}
        for idx, row in monthly_trends.iterrows()
    ]
}

# Save to JSON for dashboard
with open('analysis_results.json', 'w') as f:
    json.dump(dashboard_data, f, indent=2)

print("✅ Analysis complete! Results saved to 'analysis_results.json'")



GENERATING DASHBOARD DATA
✅ Analysis complete! Results saved to 'analysis_results.json'


In [13]:
# ==================== STEP 12: KEY RECOMMENDATIONS ====================
print("\n" + "=" * 60)
print("💡 ACTIONABLE RECOMMENDATIONS")
print("=" * 60)

print(f"""
1. 🎯 FOCUS ON {worst_condition.upper()}
   • Success rate is lowest at {condition_stats.loc[worst_condition, 'Success_Rate']:.1f}%
   • Review treatment protocols and consider specialist consultation
   
2. 💰 COST REDUCTION STRATEGY
   • {most_expensive} has highest average cost (${condition_stats.loc[most_expensive, 'Avg_Cost']:,.2f})
   • Consider preventive care programs to reduce acute cases
   
3. 📅 RESOURCE ALLOCATION
   • {len(expensive_long_stay)} patients require extended, expensive care
   • Implement early intervention programs
   
4. 🔄 REDUCE READMISSIONS
   • Current readmission rate: {readmission_rate:.1f}%
   • Strengthen discharge planning and follow-up care
   
5. 📊 LEVERAGE SUCCESSFUL TREATMENTS
   • {best_condition} shows {condition_stats.loc[best_condition, 'Success_Rate']:.1f}% success rate
   • Apply best practices to other conditions
""")

print("=" * 60)
print("Analysis complete! Ready for visualization.")
print("=" * 60)


💡 ACTIONABLE RECOMMENDATIONS

1. 🎯 FOCUS ON HEART DISEASE
   • Success rate is lowest at 67.0%
   • Review treatment protocols and consider specialist consultation

2. 💰 COST REDUCTION STRATEGY
   • Neurological has highest average cost ($20,688.89)
   • Consider preventive care programs to reduce acute cases

3. 📅 RESOURCE ALLOCATION
   • 24 patients require extended, expensive care
   • Implement early intervention programs

4. 🔄 REDUCE READMISSIONS
   • Current readmission rate: 4.3%
   • Strengthen discharge planning and follow-up care

5. 📊 LEVERAGE SUCCESSFUL TREATMENTS
   • Diabetes shows 100.0% success rate
   • Apply best practices to other conditions

Analysis complete! Ready for visualization.
